# Tramento dos dados brutos

Após a carga dos dataset a partir dos arquivos (doCarga), nesta etapa os dados sofrerão limpesa e tratamento, aonde serão retiradas "sugeiras grosseiras"

In [ ]:
def doETL():
    stopwords = set(stopwords.words('portuguese') )

In [ ]:
listaRetiraInicio = [] # ['BB','Z ', 'MC ']
listaRetiraFim    = [] # [' CC']
listaCaracteresIndesejados = ["	", # tabulação
                              "*",
                              "#",
                              "|",
                              '¥'
                             ]

In [ ]:
# retira stop words do nome dos usuários reais
def limpaTexto( entrada ):
    
    # PROBLEMA: TEM QUE VERIFICAR A LISTA DE STOPWORDS PARA QUE ELA 
    # NÃO RETIRE DESCRIÇÃO VÁLIDA DE PRODUTOS.
    # retira stopwords
    entrada = str(entrada)
    entrada = entrada.split(" ")
    saida = ""
    for item in entrada:
        if item.lower() not in stopwords:
            saida = saida + item + " "
    
    # retira acentos
    aux   = str(saida)
    nfkd  = unicodedata.normalize('NFKD', aux)
    saida = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    
    # transforma em maiúsculo, retira espaços dos extremos (TRIM), 
    saida = saida.upper().strip()
    
    # retira caracteres indesejados --> cuidado para não unir palavras separadas pelo caracter
    for item in listaCaracteresIndesejados:
        saida = saida.replace(item,"")    
    
    # retira espaços duplos dentro da string
    saida = saida.replace("  "," ")
    while saida.find('  ') != -1:
        saida = saida.replace("  "," ")
              
    # retira inicios de strings que constam na lista listaRetiraInicio
    for item in listaRetiraInicio:
        # retira espaços duplos dentro do item, pois o mesmo já foi feito no texto a pesquisar
        item = item.replace("  "," ")
        while item.find('  ') != -1:
            item = item.replace("  "," ")
            
        if saida.startswith(item):
            tamanhoItem  = len(item)
            tamanhoSaida = len(saida)            
            if tamanhoSaida > tamanhoItem:
                saida = saida[tamanhoItem:]
    
    # retira fins de strings que constam na lista listaRetiraFim
    for item in listaRetiraFim:
        # retira espaços duplos dentro do item, pois o mesmo já foi feito no texto a pesquisar
        item = item.replace("  "," ")
        while item.find('  ') != -1:
            item = item.replace("  "," ")        
        
        if saida.endswith(item):
            tamanhoItem  = len(item)
            tamanhoSaida = len(saida)
            if tamanhoSaida > tamanhoItem:
                saida = saida[:tamanhoSaida-tamanhoItem]   



    # retira espaços duplos dentro da string. Necessário pois pode ter sobrado 
    # espaço depois das alterações
    saida = saida.strip()
        
    return saida 

In [ ]:
# Somente para teste da rotina de limpa texto
resp = limpaTexto('00000lf1412214')
resp

In [ ]:
# Insere coluna "NovaDescricao" com as descrições tratadas 
novadescricao = map( limpaTexto, df_original['Desc_Catalogo'] ) 
df_original.insert( loc = 2, column = "NovaDescricao", value = list(novadescricao))#, allow_duplicates = False)
del novadescricao

In [ ]:
# Apaga linhas cuja coluna 'NovaDescricao' esteja em branco
df_original['NovaDescricao'].dropna( axis=0, inplace=True )  

In [ ]:
# Concatena 3 colunas para formar a coluna 'Categoria', que é a classificação do produto
df_original['Categoria'] = df_original['Item_Anexo_IV'].astype(str) + df_original['Prod_ST'] + df_original['Prod_FCP']

In [ ]:
# Apaga colunas que não serão usadas
df_original.drop(['Desc_Catalogo', 'NCM_NFe', 'Desc_Anexo_IV', 'NCM_Calc_Prov', 'Cor_NCM_Calc', 'NCM_Calc', 
         'Item_Anexo_IV', 'Prod_ST', 'Prod_FCP'], axis=1, inplace=True)

In [ ]:
# Renomeia as colunas do DF
df_original.columns = ('Remetente','NovaDescricao','Categoria')

## Elimina linhas inconsistentes, repetidas. Mantém a primeira delas

In [ ]:
antes = df_original.shape[0]
df_original.drop_duplicates( subset = ["NovaDescricao"], inplace = True) 
print( 'Eliminados', antes - df_original.shape[0], 'registros')
del antes

In [ ]:
# Retira categorias indesejadas
retirar = ['nannãonão','nansimnão','nansimsim']
mask = ~df_original['Categoria'].isin(retirar)
filtrado = df_original[mask].copy()
df_original = filtrado.copy()
del filtrado, retirar

In [ ]:
# Cria o DF com os dados obtidos da base a ser trabalhada, já ajustada. 
# O df_original fica com backup, durante o desenvolimento porque a base demora para ser lida.

df = df_original.copy()

# CRIA LISTA DE PALAVRAS, COM CÁLCULO DO Nº DE REPETIÇÃO - INICIO

In [ ]:
# Gera lista com todas as palavras, com repetições
# CountVectorizer retira duplicidade
total_linhas = df.shape[0]
lista_palavras = []
for i in range(0,total_linhas):
    texto = df.iloc[i, 1].split()
    tamanho_linha = len(texto)
    for j in range(0,tamanho_linha):
        lista_palavras.append( texto[j]) 
        
# Gera DF com a contagem de cada palavra
lista_palavras = pd.Series(data = lista_palavras)
df_lista_palavras = pd.DataFrame(data = lista_palavras.value_counts() )#, index = lista_palavras)
df_lista_palavras.reset_index(drop = False, inplace = True)
df_lista_palavras.columns = ['Palavra','QTD']

#libera memoria
del total_linhas, lista_palavras, texto, tamanho_linha,

In [ ]:
Lista_palavras_eliminar = []

In [ ]:
########### elimina palavras de 1 caracter

mask = (df_lista_palavras['Palavra'].str.len() == 1)
df_aux = df_lista_palavras[mask]
Lista_palavras_eliminar = Lista_palavras_eliminar + list(df_aux['Palavra'])
mask = ~(df_lista_palavras['Palavra'].str.len() == 1)
df_aux = df_lista_palavras[mask]
df_lista_palavras = df_aux.copy()

del df_aux, mask

In [ ]:
########### elimina palavras que são somente numeros, exceto os de 13 algarismos ( possíveis EAN)

# Separa possíveis códigos EAN
mask1 = df_lista_palavras.Palavra.str.len() == 13
mask2 = df_lista_palavras.Palavra.str.isdigit()
df_aux = df_lista_palavras[~(mask1 & mask2)]
# elimina números
mask2 = df_aux.Palavra.str.isdigit()
df_aux = df_aux[mask2]

Lista_palavras_eliminar = Lista_palavras_eliminar + list(df_aux['Palavra'])
mask = ~df_lista_palavras['Palavra'].isin( list(df_aux['Palavra']) )
df_aux = df_lista_palavras[mask]
df_lista_palavras = df_aux.copy()
del df_aux, mask1, mask2

In [ ]:
def EliminaPalavrasVindasDeArquivo( nome_arquivo ):
    global Lista_palavras_eliminar
    global df_lista_palavras
    df_aux = pd.read_csv(nome_arquivo, index_col = ['Indice'], sep = ';',)
    Lista_palavras_eliminar = Lista_palavras_eliminar + list(df_aux['Palavra'])
    mask = ~df_lista_palavras['Palavra'].isin( list(df_aux['Palavra']) )
    df_aux = df_lista_palavras[mask]
    df_lista_palavras = df_aux.copy()
    del df_aux, mask

## Trata palavras terminadas com ML (mililitros) - INICIO 

In [ ]:
ask1  = df_lista_palavras['Palavra'].str.endswith('ML')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

df_aux.to_csv("ETL_ML_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULA DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

EliminaPalavrasVindasDeArquivo( "ETL_ML_verificado.csv" )

## Trata palavras pela quantidade delas - INICIO 

In [ ]:
mask  = df_lista_palavras['QTD'] > 1000 
df_aux = df_lista_palavras[ mask ]

df_aux.to_csv("ETL_QUANTIDADE_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
del df_aux, mask

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

EliminaPalavrasVindasDeArquivo( "ETL_QUANTIDADE_verificado.csv" )

## Trata palavras terminadas com KG  - INICIO

In [ ]:
mask1  = df_lista_palavras['Palavra'].str.endswith('KG')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

df_aux.to_csv("ETL_KG_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

EliminaPalavrasVindasDeArquivo( "ETL_KG_verificado.csv" )

## Trata palavras terminadas com GR  - INICIO

In [ ]:
mask1  = df_lista_palavras['Palavra'].str.endswith('GR')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

df_aux.to_csv("ETL_GR_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

EliminaPalavrasVindasDeArquivo( "ETL_GR_verificado.csv" )

## Trata palavras terminadas com GRAMAS  - INICIO

In [ ]:
mask1  = df_lista_palavras['Palavra'].str.endswith('GRAMAS')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

if df_aux.shape[0] > 1:
    df_aux.to_csv("ETL_GRAMAS_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
else:
    print( 'Não há palavras terminadas com esta opcao, ou há somente uma.' )
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO.

#EliminaPalavrasVindasDeArquivo( "ETL_GRAMAS_verificado.csv" )

## Trata palavras terminadas com GRAMA  - INICIO

In [ ]:
mask1  = df_lista_palavras['Palavra'].str.endswith('GRAMA')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

if df_aux.shape[0] > 1:
    df_aux.to_csv("ETL_GRAMA_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
else:
    print( 'Não há palavras terminadas com esta opcao, ou há somente uma.' )
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

#EliminaPalavrasVindasDeArquivo( "ETL_GRAMA_verificado.csv" )

## Trata palavras terminadas com G  - INICIO

In [ ]:
mask1  = df_lista_palavras['Palavra'].str.endswith('G')
mask2  = df_lista_palavras['Palavra'].str.len() < 80
mask3  = df_lista_palavras['QTD'] > 10 
df_aux = df_lista_palavras[ mask1 & mask2 & mask3 ]

if df_aux.shape[0] > 1:
    df_aux.to_csv("ETL_G_a_verificar.csv", index = True, columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")
else:
    print( 'Não há palavras terminadas com esta opcao, ou há somente uma.' )
del df_aux, mask1, mask2, mask3

In [ ]:
# SOMENTE EXECUTE ESTA CELULAR DEPOIS DE VERIFICAR MANUALMENTE O ARQUIVO

EliminaPalavrasVindasDeArquivo( "ETL_G_verificado.csv" )

## Salva dados - inicio

In [ ]:
# Salva as palavras da base, já com as devidas retiradas de palavras
df_lista_palavras.to_csv("ETL_base_lista_palavras_corretas.csv", index = True, 
                         columns = ["Palavra", "QTD"], sep = ';', encoding = "utf-8")

df_lista_palavras.to_pickle("./ETL_base_lista_palavras_corretas.pkl")

## Elimina palavras indesejadas - INICIO

In [ ]:
palavraPermitida = df_lista_palavras['Palavra']

In [ ]:
dicPermitido = dict(zip(palavraPermitida, palavraPermitida))

In [ ]:
def verificaDescricao(frase):
    global dicPermitido
    
    resp = ''
    frase = frase.split(" ")
    for palavra in frase:
        resp = resp + " " + dicPermitido.get(palavra,'')
    resp = resp.strip()
    resp = resp.replace("  "," ")
    return resp    

In [ ]:
aux = list( map(lambda x: verificaDescricao(x), list(df_original['NovaDescricao']) ) )

In [ ]:
df_original['NovaDescricao'] = aux

In [ ]:
df_original.to_pickle("./ETL_base_pronta_para_previsao.pkl")

In [ ]:
df_original['Categoria'].value_counts()